**Adaptation of Yahoo Finance Stock Price Scraper by Debra Ray with lxml**

In [1]:
import requests
import pandas as pd
from bs4 import BeautifulSoup
from datetime import datetime, timedelta
import time
import lxml
from lxml import html

In [2]:
def format_date(date_datetime):
    date_timetuple = date_datetime.timetuple()
    date_mktime = str(int(time.mktime(date_timetuple)))
    return date_mktime

def subdomain(symbol, start, end, filter='history'):
    subdomain = f'/quote/{symbol}/history?period1={start}&period2={end}&interval=1d&filter={filter}&frequency=1d'
    return subdomain
    
def header_function(subdomain):
    hdrs =  {"authority": "finance.yahoo.com",
               "method": "GET",
               "path": subdomain,
               "scheme": "https",
               "accept": "text/html",
               "accept-encoding": "gzip, deflate, br",
               "accept-language": "en-US,en;q=0.9",
               "cache-control": "no-cache",
               "cookie": "Cookie:identifier",
               "dnt": "1",
               "pragma": "no-cache",
               "sec-fetch-mode": "navigate",
               "sec-fetch-site": "same-origin",
               "sec-fetch-user": "?1",
               "upgrade-insecure-requests": "1",
               "user-agent": "Mozilla/5.0 (Windows NT 10.0; Win64)"}
    return hdrs

In [3]:
def scrape_page(url, header):
    page = requests.get(url, headers=header)
    
    element_html = html.fromstring(page.content)
    
    table = element_html.xpath('//table')
    
    table_tree = lxml.etree.tostring(table[0], method='xml')
    
    df = pd.read_html(table_tree)
    
    return df

In [4]:
symbol = 'TSLA'

dt_start = datetime.today() - timedelta(days = 100)
dt_end = datetime.today()


price_history = []

for _ in range(45):
    start = format_date(dt_start)
    end = format_date(dt_end)
    
    sub = subdomain(symbol, start, end)
    header = header_function(sub)

    base_url = 'https://finance.yahoo.com/'
    url = base_url + sub
    price_history += scrape_page(url, header)
    dt_end = dt_start
    dt_start -= timedelta(days = 100)

In [5]:
price_history[0].head()

,Date,Open,High,Low,Close*,Adj Close**,Volume
0,"Jun 26, 2020",994.78,995.00,954.87,959.74,959.74,7977448
1,"Jun 25, 2020",954.27,985.98,937.15,985.98,985.98,9222200
2,"Jun 24, 2020",994.11,1000.88,953.14,960.85,960.85,10959600
3,"Jun 23, 2020",998.88,1012.00,994.01,1001.78,1001.78,6365300
4,"Jun 22, 2020",999.95,1008.88,990.02,994.32,994.32,6362400


In [6]:
df = pd.concat([df.set_index('Date') for df in price_history])
df.drop(df.tail(5).index, inplace = True)
df

,Open,High,Low,Close*,Adj Close**,Volume
Date,,,,,,
"Jun 26, 2020",994.78,995.00,954.87,959.74,959.74,7977448
"Jun 25, 2020",954.27,985.98,937.15,985.98,985.98,9222200
"Jun 24, 2020",994.11,1000.88,953.14,960.85,960.85,10959600
"Jun 23, 2020",998.88,1012.00,994.01,1001.78,1001.78,6365300
"Jun 22, 2020",999.95,1008.88,990.02,994.32,994.32,6362400
...,...,...,...,...,...,...
"Jul 06, 2010",20.00,20.00,15.83,16.11,16.11,6866900
"Jul 02, 2010",23.00,23.10,18.71,19.20,19.20,5139800
"Jul 01, 2010",25.00,25.92,20.27,21.96,21.96,8218800


In [7]:
df.to_csv('TSLA.csv', index = True)